Installing Dependancies:

In [18]:
pip install streamlit langchain-openai langchain-community google-api-client python-dotenv

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/c6/51/f140402202af6ce1bf747243f66415c5eb2f43ba2e2ac419a7e855d20673/streamlit-1.36.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/46/30/2118537233fa72c1d91a81f5908a7e843a6601ccc68b76838ebc4951505f/altair-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for click<9,>=7.0 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for pandas<3,>=1.3.0 from https://files.pythonhosted.org/packages/22/a5/a0b255295406ed54269814bc93723cfd1a0da63fb9aaf99e1364f07923e5/pandas-

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Importing and setting up the Environment

In [19]:
import os
import openai
from googleapiclient.discovery import build
from dotenv import load_dotenv
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

Importing the API keys from the .env File

In [20]:
# Load environment variables from .env file
load_dotenv()

# Set API keys
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
search_engine_id = os.getenv('SEARCH_ENGINE_ID')  # Ensure this is set in your .env file

In [21]:
# Initialize LangChain with OpenAI
llm = ChatOpenAI(model_name="gpt-4",api_key=openai_api_key)
prompt_template = PromptTemplate(
    input_variables=["topic", "context"],
    template="You are an expert programmer. Generate a detailed script and relevant code on {topic} using the following information for context and guidance:\n{context}"
)

In [22]:
chain = LLMChain(llm=llm, prompt=prompt_template)

Script Generation 

In [23]:
def generate_script(topic, context):
    result = chain.run(topic=topic, context=context, max_tokens=8192)
    return result.strip()

Getting Google Search Results to use as Context

In [24]:
def google_search(query):
    service = build("customsearch", "v1", developerKey=google_api_key)
    res = service.cse().list(q=query, cx=search_engine_id).execute()
    return res['items']

def fetch_search_data(topic):
    results = google_search(topic)
    context = ""
    for item in results:
        context += f"{item['title']}\n{item['snippet']}\n{item['link']}\n\n"
    return context

Combining search data and script generated to make a detailed script

In [26]:
def generate_detailed_script(topic):
    context = fetch_search_data(topic)
    script = generate_script(topic, context)
    return script

Calling functions and generating a simple web interface using streamlit

In [27]:
# Streamlit interface
st.title("Script Generator")

topic = st.text_input("Enter the topic for the script:")

if st.button("Generate Script"):
    if topic:
        with st.spinner("Generating script..."):
            script = generate_detailed_script(topic)
            st.success("Script generated successfully!")
            st.text_area("Generated Script", value=script, height=400)
    else:
        st.error("Please enter a topic to generate the script.")

# Run as streamlit run code_generator.py

2024-06-26 10:21:04.583 
  command:

    streamlit run C:\Users\rohan\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-06-26 10:21:04.584 Session state does not function when running a script without `streamlit run`
